In [1]:
# coding: utf-8 
import os
import numpy as np


import multiprocessing
from multiprocessing import Array
import time

In [2]:
def listdir_nohidden(path):
    return filter( lambda f: not f.startswith('.'), os.listdir(path))  

In [4]:
#获取目标文件夹的路径
# filedir = '/media/jerry/Sexxy/aa-alexnet-output/'

# 处理好的conv5的aa系数
filedir = '/media/jerry/Sexxy/aa-alexnet-tmp/data/'
#获取当前文件夹中的文件名称列表  
dir_1=list(listdir_nohidden(filedir))

print dir_1
# print type(dir_1)
# print len(dir_1)

['0.txt', '1.txt', '10.txt', '100.txt', '101.txt', '102.txt', '103.txt', '104.txt', '105.txt', '106.txt', '107.txt', '108.txt', '109.txt', '11.txt', '110.txt', '111.txt', '112.txt', '113.txt', '114.txt', '115.txt', '28.txt', '29.txt', '3.txt', '30.txt', '31.txt', '32.txt', '33.txt', '34.txt', '35.txt', '36.txt', '37.txt', '38.txt', '39.txt', '4.txt', '40.txt', '41.txt', '42.txt', '43.txt', '44.txt', '45.txt', '47.txt', '48.txt', '49.txt', '5.txt', '50.txt', '51.txt', '52.txt', '53.txt', '54.txt', '55.txt', '56.txt', '57.txt', '58.txt', '59.txt', '6.txt', '60.txt', '61.txt', '62.txt', '63.txt', '64.txt', '66.txt', '67.txt', '68.txt', '69.txt', '7.txt', '70.txt', '71.txt', '72.txt', '73.txt', '74.txt', '75.txt', '76.txt', '77.txt', '78.txt', '79.txt', '8.txt', '80.txt', '81.txt', '82.txt', '83.txt', '84.txt', '85.txt', '86.txt', '87.txt', '88.txt', '89.txt', '9.txt', '90.txt', '91.txt', '92.txt', '93.txt', '94.txt', '95.txt', '96.txt', '97.txt', '98.txt', '99.txt', '117.txt', '118.txt', 

In [4]:
# conv5_coeff_4d = np.zeros(256* 8* 1000* 1728)
# conv5_coeff_4d = conv5_coeff_4d.reshape(256, 8, 1000, 1728)

# conv5_coeff_3d = np.zeros(256* 1000* 1728)
# conv5_coeff_3d = conv5_coeff_3d.reshape(256, 1000, 1728)

# conv5_coeff_2d = np.zeros(256*  1728)
# conv5_coeff_2d = conv5_coeff_2d.reshape(256, 1728)

# # print conv5_coeff_4d.shape
# print conv5_coeff_3d.shape
# print conv5_coeff_2d.shape
a = np.zeros(3)
m_3d = Array('f', 256* 1000* 1728)
m_2d = Array('f', 256*  1728)
# m[0] = a[0]
print type(m_3d)
print len(m_3d)
print m_3d

<class 'multiprocessing.sharedctypes.SynchronizedArray'>
442368000
<SynchronizedArray wrapper for <multiprocessing.sharedctypes.c_float_Array_442368000 object at 0x7f410c011c20>>


In [5]:
def worker(num1, num2, arr_3d, arr_2d):

    for i in range(num1, num2):
        print i
        base = filedir + dir_1[i]
        d1 = dir_1[i].split(".", 1)  ## 分割符，分割次数
        kernel_num = int(d1[0])
        
    #     print base
    #     print d1
        print 'kernel_num: ', kernel_num
        b = np.loadtxt(base, dtype=np.float32)
        print b.shape

        c_3d = b.reshape(8, 1000, 1728)
    #     conv5_coeff_4d[kernel_num] = c_3d
        # print c_3d.shape ## (8, 1000, 1728)
        c_3d = np.abs(c_3d) ## 取绝对值
        d_2d = np.mean(c_3d, 0) ## 按照 8 求平均 (1000, 1728)
#         print d_2d
        m_tmp2d = d_2d.reshape(1000* 1728)
        for j in range(1000* 1728):
            m_3d[kernel_num*1000* 1728 + j] = m_tmp2d[j]
            
#         conv5_coeff_3d[kernel_num] = d_2d ## 存入3维数组对应的kernel
 
        e_1d= np.mean(d_2d, 0) ## (1728,)
    
        for k in range(1728):
            arr_2d[kernel_num*1728 + k] = e_1d[k]
        

        print 'i: ', i, 'Done'

In [6]:
def multi_process():
    p1 = multiprocessing.Process(target = worker, args = (0,64,m_3d, m_2d))
    p2 = multiprocessing.Process(target = worker, args = (64,128,m_3d, m_2d))
    p3 = multiprocessing.Process(target = worker, args = (128,192,m_3d, m_2d))
    p4 = multiprocessing.Process(target = worker, args = (192,256,m_3d, m_2d))

#     p1 = multiprocessing.Process(target = worker, args = (0, 1, m_3d, m_2d))
#     p2 = multiprocessing.Process(target = worker, args = (64,65,m_3d, m_2d))
#     p3 = multiprocessing.Process(target = worker, args = (128,129,m_3d, m_2d))
#     p4 = multiprocessing.Process(target = worker, args = (192,193,m_3d, m_2d))


    p1.start()
    p2.start()
    p3.start()
    p4.start()
    
    p1.join()
    p2.join()
    p3.join()
    p4.join()
    
    
    # print("The number of CPU is:" + str(multiprocessing.cpu_count()))
    # for p in multiprocessing.active_children():
    #     print("child   p.name:" + p.name + "\tp.id" + str(p.pid))
    # print "END!!!!!!!!!!!!!!!!!"

In [7]:
multi_process()


conv5_coeff_3d = np.zeros(256* 1000* 1728)
conv5_coeff_2d = np.zeros(256*  1728)

for j in range(256 *1000* 1728):
     conv5_coeff_3d[j] = m_3d[j]
print conv5_coeff_3d.reshape(256, 1000, 1728)

for j in range(256 * 1728):
     conv5_coeff_2d[j] = m_2d[j]

conv5_coeff_3d = conv5_coeff_3d.reshape(256, 1000, 1728)
conv5_coeff_2d = conv5_coeff_2d.reshape(256, 1728)


# print conv5_coeff_2d[0]
# print conv5_coeff_2d[7]
# print conv5_coeff_2d[137]
# print conv5_coeff_2d[198]

np.save('/home/jerry/data-alexnet/npy/c5_3d.npy', conv5_coeff_3d)
np.save('/home/jerry/data-alexnet/npy/c5_2d.npy', conv5_coeff_2d)
print 'All Process Done!'

0
kernel_num:  0
64
kernel_num:  7
128
kernel_num:  137
192
kernel_num:  198
(13824000,)
(13824000,)
(13824000,)
(13824000,)
i:  0 Done
1
kernel_num:  1
i:  192 Done
193
kernel_num:  199
i:  64 Done
65
kernel_num:  70
i:  128 Done
129
kernel_num:  138
(13824000,)
i:  1 Done
2
kernel_num:  10
(13824000,)
(13824000,)
(13824000,)
i:  193 Done
194
kernel_num:  2
i:  65 Done
66
kernel_num:  71
i:  129 Done
130
kernel_num:  139
(13824000,)
i:  2 Done
3
kernel_num:  100
(13824000,)
(13824000,)
i:  130 Done
131
kernel_num:  14
i:  66 Done
67
kernel_num:  72
(13824000,)
i:  194 Done
195
kernel_num:  20
(13824000,)
i:  131 Done
132
kernel_num:  140
(13824000,)
i:  195 Done
196
kernel_num:  200
(13824000,)
i:  3 Done
4
kernel_num:  101
(13824000,)
i:  67 Done
68
kernel_num:  73
(13824000,)
i:  132 Done
133
kernel_num:  141
(13824000,)
i:  196 Done
197
kernel_num:  201
(13824000,)
i:  4 Done
5
kernel_num:  102
(13824000,)
i:  68 Done
69
kernel_num:  74
(13824000,)
i:  133 Done
134
kernel_num:  142

(13824000,)
i:  43 Done
44
kernel_num:  50
(13824000,)
i:  174 Done
175
kernel_num:  181
(13824000,)
i:  110 Done
111
kernel_num:  13
(13824000,)
i:  239 Done
240
kernel_num:  242
(13824000,)
i:  44 Done
45
kernel_num:  51
(13824000,)
i:  175 Done
176
kernel_num:  182
(13824000,)
i:  111 Done
112
kernel_num:  130
(13824000,)
i:  240 Done
241
kernel_num:  243
(13824000,)
i:  45 Done
46
kernel_num:  52
(13824000,)
i:  176 Done
177
kernel_num:  183
(13824000,)
i:  112 Done
113
kernel_num:  131
(13824000,)
i:  241 Done
242
kernel_num:  244
(13824000,)
i:  46 Done
47
kernel_num:  53
(13824000,)
i:  177 Done
178
kernel_num:  184
(13824000,)
i:  113 Done
114
kernel_num:  132
(13824000,)
i:  242 Done
243
kernel_num:  245
(13824000,)
i:  47 Done
48
kernel_num:  54
(13824000,)
i:  178 Done
179
kernel_num:  185
(13824000,)
i:  114 Done
115
kernel_num:  133
(13824000,)
i:  243 Done
244
kernel_num:  246
(13824000,)
i:  48 Done
49
kernel_num:  55
(13824000,)
i:  179 Done
180
kernel_num:  186
(138240

In [8]:
# test = np.zeros(256 *1000 * 1728)
# for j in range(256 *1000* 1728):
#      test[j] = m_3d[j]
# print test.reshape(256, 1000, 1728)
# if (conv5_coeff_3d.all() == 0): print 1
# print conv5_coeff_3d[0]

In [9]:
conv5_coeff_3d = np.zeros(256* 1000* 1728)
conv5_coeff_2d = np.zeros(256*  1728)

# for j in range(256 *1000* 1728):
#      conv5_coeff_3d[j] = m_3d[j]
# print conv5_coeff_3d.reshape(256, 1000, 1728)

for j in range(256 * 1728):
     conv5_coeff_2d[j] = m_2d[j]

# conv5_coeff_3d = conv5_coeff_3d.reshape(256, 1000, 1728)
conv5_coeff_2d = conv5_coeff_2d.reshape(256, 1728)

In [10]:
print conv5_coeff_2d[0]
print conv5_coeff_2d[7]
print conv5_coeff_2d[137]
print conv5_coeff_2d[198]

[1.98067212e-07 2.22565717e-07 3.29617137e-07 ... 1.67751949e-07
 3.37968196e-08 1.01581051e-07]
[3.56958793e-07 2.94522181e-07 1.74524317e-07 ... 3.12615924e-07
 2.74989048e-07 1.52651992e-07]
[1.92578625e-07 2.18143356e-07 1.19792617e-07 ... 1.07761878e-07
 8.68174155e-08 6.12747826e-08]
[6.12737097e-08 1.43421687e-07 1.21967318e-07 ... 2.09103050e-07
 1.65511594e-07 2.41402347e-07]


In [11]:
# np.save('/home/jerry/data-alexnet/npy/c5_3d.npy', conv5_coeff_3d)
np.save('/home/jerry/data-alexnet/npy/c5_2d.npy', conv5_coeff_2d)

In [12]:
a = np.load('/home/jerry/data-alexnet/npy/c5_2d.npy')

print a[0]
print a[7]
print a[137]
print a[198]

[1.98067212e-07 2.22565717e-07 3.29617137e-07 ... 1.67751949e-07
 3.37968196e-08 1.01581051e-07]
[3.56958793e-07 2.94522181e-07 1.74524317e-07 ... 3.12615924e-07
 2.74989048e-07 1.52651992e-07]
[1.92578625e-07 2.18143356e-07 1.19792617e-07 ... 1.07761878e-07
 8.68174155e-08 6.12747826e-08]
[6.12737097e-08 1.43421687e-07 1.21967318e-07 ... 2.09103050e-07
 1.65511594e-07 2.41402347e-07]
